In [ ]:
import sys
from PyQt5.QtCore import QDate, QDateTime, QTimer, QTime
from PyQt5 import QtCore, QtGui, QtWidgets, uic
from PyQt5.QtWidgets import QMainWindow, QLabel, QPushButton, QApplication, QFileDialog, QMessageBox, QDialog, QFrame
from PyQt5.uic import loadUi
import os
import numpy as np
import torch
import tensorflow.keras as tf
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import bone
from datetime import datetime
import cv2
import matplotlib.pyplot as plt

# weight path ----------------------------------------------------------------------
model_path = './weight/model.pt'
tjnet_path = './weight/tjnet24.h5'

# yolo = torch.load(model_path, map_location='cpu')

now = datetime.now()
formattedDate = now.strftime("%Y%m%d_%H%M%S")
filename = formattedDate +'.jpg'
save_path = './img_save/' + filename


#  form ---------------------------------------------------------------------------
form_secondwindow =uic.loadUiType("dashboard1.ui")[0]


# ui -------------------------------------------------------------------------
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(1000, 800)
        icon = QtGui.QIcon()
        icon.addPixmap(QtGui.QPixmap("./boneage.png"), QtGui.QIcon.Normal, QtGui.QIcon.Off)
        MainWindow.setWindowIcon(icon)
        MainWindow.setStyleSheet("background-color: rgb(247, 242, 231);\n"
"border-color: rgb(121, 122, 126);")
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.frame = QtWidgets.QFrame(self.centralwidget)
        self.frame.setGeometry(QtCore.QRect(25, 20, 950, 700))
        self.frame.setStyleSheet("color: rgb(255, 255, 255);")
        self.frame.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.frame.setFrameShadow(QtWidgets.QFrame.Sunken)
        self.frame.setLineWidth(10)
        self.frame.setObjectName("frame")
        self.time = QtWidgets.QDateTimeEdit(self.frame)
        self.time.setGeometry(QtCore.QRect(10, 10, 194, 22))
        self.time.setBaseSize(QtCore.QSize(0, 0))
        font = QtGui.QFont()
        font.setFamily("한컴 말랑말랑 Regular")
        font.setPointSize(10)
        font.setBold(False)
        font.setWeight(50)
        self.time.setFont(font)
        self.time.setStyleSheet("color: rgb(047, 079, 079);")
        self.time.setWrapping(False)
        self.time.setFrame(False)
        self.time.setAlignment(QtCore.Qt.AlignCenter)
        self.time.setReadOnly(True)
        self.time.setButtonSymbols(QtWidgets.QAbstractSpinBox.NoButtons)
        self.time.setDateTime(QDateTime.currentDateTime())
        self.time.setObjectName("time")
        self.progressBar = QtWidgets.QProgressBar(self.frame)
        self.progressBar.setEnabled(False)
        self.progressBar.setGeometry(QtCore.QRect(570, 660, 290, 23))
        font = QtGui.QFont()
        font.setFamily("한컴 말랑말랑 Regular")
        font.setPointSize(12)
        self.progressBar.setFont(font)
        self.progressBar.setCursor(QtGui.QCursor(QtCore.Qt.ArrowCursor))
        self.progressBar.setMouseTracking(False)
        self.progressBar.setTabletTracking(False)
        self.progressBar.setFocusPolicy(QtCore.Qt.NoFocus)
        self.progressBar.setContextMenuPolicy(QtCore.Qt.DefaultContextMenu)
        self.progressBar.setAcceptDrops(False)
        self.progressBar.setToolTipDuration(-1)
        self.progressBar.setLayoutDirection(QtCore.Qt.LeftToRight)
        self.progressBar.setAutoFillBackground(False)
        self.progressBar.setStyleSheet("background-color: qlineargradient(spread:pad, x1:0, y1:0, x2:0, y2:0, stop:0 rgba(0, 0, 0, 0), stop:1 rgba(121, 122, 126));")
        self.progressBar.setMaximum(100)
        self.progressBar.setProperty("value", 24)
        self.progressBar.setTextVisible(True)
        self.progressBar.setOrientation(QtCore.Qt.Horizontal)
        self.progressBar.setInvertedAppearance(False)
        self.progressBar.setTextDirection(QtWidgets.QProgressBar.TopToBottom)
        self.progressBar.setObjectName("progressBar")
        self.title = QtWidgets.QLabel(self.frame)
        self.title.setGeometry(QtCore.QRect(100, 180, 741, 121))
        font = QtGui.QFont()
        font.setFamily("Algerian")
        font.setPointSize(52)
        font.setBold(True)
        font.setWeight(75)
        self.title.setFont(font)
        self.title.setStyleSheet("color: qlineargradient(spread:pad, x1:0, y1:0, x2:0, y2:0, stop:0 rgba(0, 0, 0, 0), stop:1 rgba(121, 122, 126));")
        self.title.setAlignment(QtCore.Qt.AlignCenter)
        self.title.setIndent(-1)
        self.title.setTextInteractionFlags(QtCore.Qt.NoTextInteraction)
        self.title.setObjectName("title")
        self.next_button = QtWidgets.QPushButton(self.frame)
        self.next_button.setGeometry(QtCore.QRect(570, 580, 291, 61))
        font = QtGui.QFont()
        font.setFamily("Calibri")
        font.setPointSize(14)
        font.setBold(True)
        font.setWeight(75)
        self.next_button.setFont(font)
        self.next_button.setStyleSheet("background-color:rgb(216, 211, 205);\n"
"border:none;\n"
"border-bottom: 2px solid rgb(35, 35, 35);\n"
"color: rgb(50, 50, 50);\n"
"border-bottom-right-radius: 15px;\n"
"border-bottom-left-radius: 15px;\n"
"border-top-right-radius: 15px;\n"
"border-top-left-radius: 15px;\n"
"\n"
"QPushButton#pushButton:pressed{\n"
"    padding-left:5px;\n"
"    padding-top:5px;\n"
"    background-color: rgb(194, 194, 194);\n"
"}")
        icon1 = QtGui.QIcon()
        icon1.addPixmap(QtGui.QPixmap(":/newPrefix/dashboard2.ui"), QtGui.QIcon.Selected, QtGui.QIcon.On)
        self.next_button.setIcon(icon1)
        self.next_button.setAutoDefault(True)
        self.next_button.setDefault(False)
        self.next_button.setObjectName("next_button")
        self.bone_img = QtWidgets.QLabel(self.frame)
        self.bone_img.setGeometry(QtCore.QRect(380, 300, 681, 321))
        self.bone_img.setStyleSheet("image: url(boneage.png);\n"
"background-color: qlineargradient(spread:pad, x1:0, y1:0, x2:0, y2:0, stop:0 rgba(0, 0, 0, 0), stop:1 rgba(255, 255, 255, 0));")
        self.bone_img.setText("")
        self.bone_img.setObjectName("bone_img")
        self.bone_img.raise_()
        self.title.raise_()
        self.time.raise_()
        self.progressBar.raise_()
        self.next_button.raise_()
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 1000, 26))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        # self.progressBar.valueChanged['int'].connect(self.progressbar_change)
        self.next_button.clicked.connect(self.progressbar_change)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Bone Age"))
        self.time.setDisplayFormat(_translate("MainWindow", "yyyy-MM-dd  hh:mm"))
        self.title.setText(_translate("MainWindow", "<html><head/><body><p align=\"center\"><span style=\" color:#797a7e;\">Bone Age predictor</span></p></body></html>"))
        self.next_button.setText(_translate("MainWindow", "Loading, click"))

    def progressbar_change(self):
        self.next_button.setText("Loading")
        while self.next_button.text() == "Loading":
            self.progressBar.setValue(np.random.randint(40,70))
            try :
                global tjnet_path
                global model_path
                global tjnet
                tjnet = tf.models.load_model(tjnet_path, compile=False)
                self.progressBar.setValue(np.random.randint(70,80))
                global yolo
                yolo = torch.load(model_path, map_location='cpu')

                self.next_button.setText("Completed, Strat")
                if self.next_button.text() == "Completed, Strat":
                    self.progressBar.setValue(100)
                    MainWindow.close()
                    widget.show()
                    # self.progressBar.valueChanged.connect(self.next_page)

                    print('True')

                else : print('progress wrong')

            except : 
                print('ERROR > check path.')
                break
    
    def next_page(self):
        MainWindow.close()
        # se = secondwindow()
        # se.exec()
        widget.show()
#-----------------------------------------------------------------------------------------------------
class secondwindow(QDialog, QFrame, form_secondwindow):
    def __init__(self):
        super(secondwindow, self).__init__()
        loadUi("dashboard1.ui", self)
        font = QtGui.QFont()
        font.setFamily("한컴 말랑말랑 Regular")
        font.setPointSize(10)
        font.setBold(False)
        font.setWeight(50)
        self.time2.setFont(font)
        self.time2.setStyleSheet("color: rgb(047, 079, 079);")
        self.time2.setDateTime(QDateTime.currentDateTime())
        self.image_frame = QLabel(self)
             # 이미지 불러오기
    def filedialog_open(self):
        fname = QFileDialog.getOpenFileName(self, 'Open File', '',
                                            'All File(*);; Image File(*.png *.jpg)')
        
        if fname[0]:
            # QPixmap 객체
            self.pixmap = QtGui.QPixmap(fname[0])            
            self.pixmap = self.pixmap.scaled(481,621) # 이미지 스케일 변화
            self.image_frame.move(50,50) # 시작위치
            self.image_frame.setPixmap(self.pixmap)  # 이미지 세팅
            self.image_frame.setContentsMargins(0,0,0,0)
            self.image_frame.resize(481,621)  # 프레임 스케일
            
#             이미지의 크기에 맞게 Resize
#             self.resize(481,621)    # 전체화면 스케일
            
        else:
            QMessageBox.about(self, 'Warning', '파일을 선택하지 않았습니다.')
        
        openpath = fname[0]
        openimg = bone.read_img(openpath)
        mask = bone.make_mask(openimg)
        masked = bone.cut_mask(openimg, mask)
        rotated_img = bone.img_rotation(masked)
        bone_img = bone.Decomposing(rotated_img,60,55,50,25)
        cv2.imwrite(save_path, bone_img)
        
        return fname[0]

    # Female, Male 버튼
    def female(self):
        global gender, gender_text
        gender = 0
        gender_text = 'Female'
    
    def male(self):
        global gender, gender_text
        gender = 1
        gender_text = 'Male'


    # # 다음페이지로 
    def gotonextpage(self):
        widget.setCurrentIndex(widget.currentIndex()+1)
        
        global yolo
        global result
        crops, yoloimg, result = bone.yolo_crop_img(save_path, yolo)
        h,w,c = yoloimg.shape

        qImg = QtGui.QImage(yoloimg, w, h, w*c, QtGui.QImage.Format_RGB888)
        th.yolo_img = QtGui.QPixmap.fromImage(qImg)
        th.yolo_img = th.yolo_img.scaled(411,521)
        th.yolo_frame.move(40,70)
        th.yolo_frame.setPixmap(th.yolo_img)  # 이미지 세팅
        th.yolo_frame.setContentsMargins(0,0,0,0)
        th.yolo_frame.resize(411,521)  # 프레임 스케일 

        X = bone.out_crop_img(crops, gender)
        prediction_BA = bone.predict_zscore(X, tjnet)
        prediction_BA = prediction_BA.round(2)

        th.input_gender2.setText(gender_text) 
        th.input_gender2.setReadOnly(True)
        th.pred.setText(f'{prediction_BA}') 
        th.pred.setReadOnly(True)
        

#-----------------------------------------------------------------------------------------------------
class thirdwindow(QDialog):
    def __init__(self):
        super(thirdwindow, self).__init__()
        loadUi("dashboard2.ui", self)
        font = QtGui.QFont()
        font.setFamily("한컴 말랑말랑 Regular")
        font.setPointSize(10)
        font.setBold(False)
        font.setWeight(50)
        self.time3.setFont(font)
        self.time3.setStyleSheet("color: rgb(047, 079, 079);")
        self.time3.setDateTime(QDateTime.currentDateTime())
        self.before_button.clicked.connect(self.gotobeforepage)
        self.yolo_frame = QLabel(self)
    

    # 다음페이지로 
    def gotobeforepage(self):
        widget.setCurrentIndex(widget.currentIndex()-1)

#-----------------------------------------------------------------------------------------------------

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    widget = QtWidgets.QStackedWidget()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    
    # app.exec_()
    # 화면전환용 위젯 생성
    widget = QtWidgets.QStackedWidget()
    
    # 레이아웃 인스턴스 생성
    se = secondwindow()
    th = thirdwindow()
    
    # 위젯 추가
    widget.addWidget(se)
    widget.addWidget(th)
    
    # 프로그램 화면
    widget.setFixedHeight(800)
    widget.setFixedWidth(1000)
    
    MainWindow.show()

    sys.exit(app.exec_())
    # MainWindow.close()
